In [1]:
import transformer_lens
import torch

In [96]:
# Define a simple prompt
prompt = ["Once upon a time, there was a tiny robot who loved to tell stories.","Once upon a time, stories."]

# Tokenize the prompt
tokens = model.tokenizer(prompt, return_tensors="pt")

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [13]:
tokens['input_ids'].shape

torch.Size([2, 16])

In [64]:
# Define a hook function to capture MLP activations
mlp_activations = []

def capture_mlp_activations(module, ip, output):
    mlp_activations.append(output.detach().cpu())

In [50]:
# Load the pre-trained "tiny-stories-1L-21M" model
model_name = "tiny-stories-1L-21M"  # Replace this with the actual model name
model = transformer_lens.HookedTransformer.from_pretrained(model_name)

Loaded pretrained model tiny-stories-1L-21M into HookedTransformer


In [51]:
# singly Transformer Block
model.cfg.n_layers

1

In [53]:
model.blocks[0].mlp

MLP(
  (hook_pre): HookPoint()
  (hook_post): HookPoint()
)

In [54]:
# Register hooks to capture MLP activations at each layer
# Assuming the MLP layers are named "mlp" in the model (adjust if necessary)
for layer_index in range(model.cfg.n_layers):
    model.blocks[layer_index].mlp.register_forward_hook(capture_mlp_activations)

In [60]:
tokens['input_ids'].reshape(-1).shape

torch.Size([32])

In [83]:
# Pass the prompt through the model

with torch.no_grad():
    model(tokens['input_ids'].reshape(-1))

# Print out the collected MLP activations
for i, activation in enumerate(mlp_activations):
    print(f"Layer {i} MLP activations: {activation.shape}")

Layer 0 MLP activations: torch.Size([1, 32, 1024])
Layer 1 MLP activations: torch.Size([1, 32, 1024])


Layer 0 MLP activations: torch.Size([1, 32, 1024])


In [84]:
active1 = mlp_activations[0].squeeze(0)
active2 = mlp_activations[0].squeeze(0)

In [85]:
active1[0] - active2[0]

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [78]:
model.to('cuda')

Moving model to device:  cuda


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resi

In [81]:
dir(model.blocks[0].mlp)

['T_destination',
 'W_in',
 'W_out',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_pe

In [82]:
tokens

{'input_ids': tensor([[7454, 2402,  257,  640,   11,  612,  373,  257, 7009, 9379,  508, 6151,
          284, 1560, 3923,   13],
        [7454, 2402,  257,  640,   11,  612,  373,  257, 7009, 9379,  508, 6151,
          284, 1560, 3923,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [88]:
for t in tokens['input_ids']:
    print(t)

tensor([7454, 2402,  257,  640,   11,  612,  373,  257, 7009, 9379,  508, 6151,
         284, 1560, 3923,   13])
tensor([7454, 2402,  257,  640,   11,  612,  373,  257, 7009, 9379,  508, 6151,
         284, 1560, 3923,   13])


## Activations for Gender-sensitive words

In [89]:
import pandas as pd

In [90]:
gendered_data = pd.read_csv('gender_data.csv')

In [91]:
gendered_data.head()

,Word,Gender,Category
0,Abigail,Female,Name
1,Adeline,Female,Name
2,Admiral,Male,Common Noun
3,Adventuress,Female,Common Noun
4,Alethea,Female,Name


In [95]:
gendered_data['Word'].value_counts(), gendered_data['Word'].value_counts().max()

(Word
 Abigail    1
 Mermaid    1
 Mother     1
 Morgan     1
 Monk       1
           ..
 Ezra       1
 Ewe        1
 Everley    1
 Evelyn     1
 Zoe        1
 Name: count, Length: 312, dtype: int64,
 1)